In [1]:
import folium
import pandas as pd
#import pdvega

import geopandas as gpd
from folium.plugins import MeasureControl, Fullscreen # for ruler in maps
from folium.features import DivIcon
import numpy as np
import geopy

%matplotlib inline
locator = geopy.Nominatim(user_agent="myGeocoder")

# Load Data

In [2]:
df_producers = pd.read_excel("./data/producers.xlsx")
df_producers

,Name,Layer,Description,Address,Phone,email,LON,LAT,Available,Total_Produce,Status,Icon,Color
0,Max Mustermann,Masken Verfügbar,Max Mustermann,"Im Leingarten 6, 32549 Bad Oeynhausen",05731 - 123456789,ich@Beispiel.de,NaN,NaN,2,10,working,fa-user,green


In [3]:
df_consumers = pd.read_excel("./data/consumers.xlsx")
df_consumers

,Name,Layer,Description,Address,Phone,email,LON,LAT,Status,Demand,URL,Icon,Color
0,Beispielklinik,Bedarf,Krankenhaus,"Georgstraße 11, 32545 Bad Oeynhausen",05731 - xxxx,info@hdzxxx.de,NaN,NaN,Bedarf an Masken,100,www.beispielkrankenhaus.de,fa-ambulance,red
1,Beispielpraxis,Bedarf,Arztpraxis,"Eidinghausener Str. 123, 32549 Bad Oeynhausen",05731 - xxxx,info@beispiel.de,NaN,NaN,Bedarf an Masken,10,www.beispielpraxis.de,fa-stethoscope,red


In [4]:
df_suppliers = pd.read_excel("./data/suppliers.xlsx")
df_suppliers

,Name,Layer,Description,Address,Phone,email,LON,LAT,Status,URL,Icon,Color
0,Max Mustermann,Material,Material Verfügbar,"Kanalstraße 1, 32545 Bad Oeynhausen",05731-xxxx,max@mustermann.de,NaN,NaN,Habe Baumwolle,NaN,fa-briefcase,blue


# Create Basemap


In [5]:
start_coord = [52.2035, 8.80445]

m = folium.Map(
    location=start_coord,
    zoom_start=13,
    tiles='openstreetmap',
    control_scale=True,
)

# Tile layers
folium.TileLayer('Stamen Terrain', attr='terrain').add_to(m)
folium.TileLayer('openstreetmap', attr='osm').add_to(m)
folium.TileLayer('cartodbpositron', attr='positron').add_to(m)

In [6]:
layers = {}

for l in df_consumers.Layer.unique():
    layers[l] = folium.FeatureGroup(name=l)
    layers[l].add_to(m)
    print(l)
    
for l in df_producers.Layer.unique():
    layers[l] = folium.FeatureGroup(name=l)
    layers[l].add_to(m)
    print(l)
    
for l in df_suppliers.Layer.unique():
    layers[l] = folium.FeatureGroup(name=l)
    layers[l].add_to(m)
    print(l)



Bedarf
Masken Verfügbar
Material


In [7]:
# Layer and Measurement Controls
folium.LayerControl(collapsed=False).add_to(m)  # now you can turn off and on layers
# folium.plugins.MeasureControl().add_to(m)
folium.plugins.Fullscreen().add_to(m)
folium.plugins.LocateControl().add_to(m)  # geolocation button (GPS) ... probably only available in higher versions
folium.plugins.MiniMap(tile_layer='Stamen Terrain', zoom_level_offset=-5).add_to(m)
folium.plugins.MousePosition(position="bottomleft").add_to(m)


# Consumer Layer

In [8]:
popup_template_consumers = """

<table width=250>
    <tr>
        <td>
        <!-- left side of popup-->
            <h4>{0}</h4>
            <p>{1}<br>
            <a href="{2}">{2}</a></p>
            <hr>
            <p>Bedarf: {3} Masken</p>

        </td>  
        <td>
        <!-- right side of popup (chart preview)-->
        </td>
    <tr>
</table>
"""

In [9]:
# create markers
for i, poi in df_consumers.iterrows():

    #vegalite_plot = df_head.reset_index().vgplot.scatter(x="Time", y=obs.label)

    location = locator.geocode(poi.Address)
    lon, lat = location.longitude, location.latitude
    
    print("{} ({}) --> {}{}".format(poi.Address, poi.Name, lon, lat))
    
    #if poi.Status == "Closed": 
    #    markercolor = "lightgray" 
    #    iconcolor = "#ff9999"
    #else: 
    markercolor = "red"
    iconcolor = "white"

    # Create Marker
    marker = folium.Marker([lat, lon], 
              tooltip=poi.Name,
              popup = popup_template_consumers.format(poi.Name, poi.Description, poi.URL, poi.Demand),
              icon = folium.map.Icon(color=markercolor, icon_color=iconcolor, icon=poi.Icon, prefix="fa")
             )
    
    marker.add_to(layers[poi.Layer])
    

    # Label layer für alle Messstellen -------------------------------------
    #labmarker = folium.map.Marker(coord,
    #                                  icon=DivIcon(
    #                                      icon_size=(30,30),
    #                                      icon_anchor=(11,24),
    #                                      html='<div style="font-size: 9pt" align="center">{}</div>'.format(obs.label)))

    # add marker / label to correct layer
    #for group in groups.keys():
    #    if obs.label in groups[group]:
    #        marker.add_to(layer[group])
    #        labmarker.add_to(layer[group])

Georgstraße 11, 32545 Bad Oeynhausen (Beispielklinik) --> 8.794459652.1924794
Eidinghausener Str. 123, 32549 Bad Oeynhausen (Beispielpraxis) --> 8.798878252.2225905


# Producer Layer

In [10]:
popup_template_producers = """

<table width=250>
    <tr>
        <td>
        <!-- left side of popup-->
            <h4>{0}</h4>
            {3}
            <p>{1}<br>
            <hr>
            <p>Verfügbar: {2} Masken</p>

        </td>  
        <td>
        <!-- right side of popup (chart preview)-->
        </td>
    <tr>
</table>
"""

In [11]:
# create markers
for i, poi in df_producers.iterrows():

    #vegalite_plot = df_head.reset_index().vgplot.scatter(x="Time", y=obs.label)

    location = locator.geocode(poi.Address)
    lon, lat = location.longitude, location.latitude
    
    print("{} ({}) --> {}{}".format(poi.Address, poi.Name, lon, lat))
    
    #if poi.Status == "Closed": 
    #    markercolor = "lightgray" 
    #    iconcolor = "#ff9999"
    #else: 
    markercolor = "green"
    iconcolor = "white"

    # Create Marker
    marker = folium.Marker([lat, lon], 
              tooltip=poi.Name,
              popup = popup_template_producers.format(poi.Name, poi.Description, poi.Available, poi.Phone),
              icon = folium.map.Icon(color=markercolor, icon_color=iconcolor, icon=poi.Icon, prefix="fa")
             )
    
    marker.add_to(layers[poi.Layer])
    

    # Label layer für alle Messstellen -------------------------------------
    #labmarker = folium.map.Marker(coord,
    #                                  icon=DivIcon(
    #                                      icon_size=(30,30),
    #                                      icon_anchor=(11,24),
    #                                      html='<div style="font-size: 9pt" align="center">{}</div>'.format(obs.label)))

    # add marker / label to correct layer
    #for group in groups.keys():
    #    if obs.label in groups[group]:
    #        marker.add_to(layer[group])
    #        labmarker.add_to(layer[group])

Im Leingarten 6, 32549 Bad Oeynhausen (Max Mustermann) --> 8.798410652.2187587


# Supplier Layer

In [12]:
popup_template_suppliers = """

<table width=250>
    <tr>
        <td>
        <!-- left side of popup-->
        
            <h4>{0}</h4>
            <p>{1}<br>
            {2}
            
        </td>  
        <td>
        <!-- right side of popup (chart preview)-->
        </td>
    <tr>
</table>
"""

In [13]:
# create markers
for i, poi in df_suppliers.iterrows():

    #vegalite_plot = df_head.reset_index().vgplot.scatter(x="Time", y=obs.label)

    location = locator.geocode(poi.Address)
    lon, lat = location.longitude, location.latitude
    
    print("{} ({}) --> {}{}".format(poi.Address, poi.Name, lon, lat))
    
    #if poi.Status == "Closed": 
    #    markercolor = "lightgray" 
    #    iconcolor = "#ff9999"
    #else: 
    markercolor = "blue"
    iconcolor = "white"

    # Create Marker
    marker = folium.Marker([lat, lon], 
              tooltip=poi.Name,
              popup = popup_template_suppliers.format(poi.Name, poi.Description, poi.Status, poi.Phone),
              icon = folium.map.Icon(color=markercolor, icon_color=iconcolor, icon=poi.Icon, prefix="fa")
             )
    
    marker.add_to(layers[poi.Layer])
    

    # Label layer für alle Messstellen -------------------------------------
    #labmarker = folium.map.Marker(coord,
    #                                  icon=DivIcon(
    #                                      icon_size=(30,30),
    #                                      icon_anchor=(11,24),
    #                                      html='<div style="font-size: 9pt" align="center">{}</div>'.format(obs.label)))

    # add marker / label to correct layer
    #for group in groups.keys():
    #    if obs.label in groups[group]:
    #        marker.add_to(layer[group])
    #        labmarker.add_to(layer[group])

Kanalstraße 1, 32545 Bad Oeynhausen (Max Mustermann) --> 8.798457452.2063023


 # Finalize

In [14]:
m.save("index.html")

In [15]:
m